# Data Cleaning for HKTVMall Alcohol
#- delete the records 
   * stock_status = 'Not found' - done
   * Price contain "全店至少 - done
   
#- band : band and description 
#- price: 
   * for the price is start with '$', remove{'$',',') 
   
   * searchfor = {'原:$','原: $','原價$','原價 $','原價:$','原價：$','原價:','原價','RSP$',' | $','！！！','！！','！'] to find the original price 
   * for the price is not start with '$', then price is replaced by promotion, and promotion will be set to zero

#- promotion : remove{'$',',')   

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pycnnum as cn # convert chinese number to arabic number
import os # take a look on the dataset
import time
import re

In [2]:
# Function to convert a list to string
def listToString(s):   
    # initialize an empty string
    str1 = "" 
    
    # traverse in the string  
    for ele in s: 
        str1 += ele  
    
    # return string  
    return str1 

# Returns length of string
def findLen(str):
    counter = 0
    while str[counter:]:
        counter += 1
    return counter

def mid(s, offset, amount):
    return s[offset-1:offset+amount-1]

def left(aString, howMany):
    if howMany <1:
        return ''
    else:
        return aString[:howMany]

def right(aString, howMany):
    if howMany <1:
        return ''
    else:
        return aString[-howMany:]

# find substring and position
def find_str(full, sub):
    index = 0
    sub_index = 0
    position = -1
    for ch_i,ch_f in enumerate(full) :
        #print (ch_f.lower(), sub[sub_index].lower())
        if ch_f.lower() != sub[sub_index].lower():
            position = -1
            sub_index = 0
        if ch_f.lower() == sub[sub_index].lower():
            if sub_index == 0 :
                position = ch_i

            if (len(sub) - 1) <= sub_index :
                break
            else:
                sub_index += 1

    return position

In [3]:
#Define Variable
#Define InDir, OutDir, Regions and Filename
extract_product="alcohol"
category="vodka"
#DateStr='20211212'
DateStr=time.strftime("%Y%m%d")

InProductsFile = "Input/HKTVMall_ProdInfo_"+extract_product+"_"+category+"_"+DateStr+".csv"
OutProductsFile = "output/HKTVMall_ProdInfo_"+extract_product+"_"+category+"_"+DateStr+"_clean.csv" 

In [4]:
# import the dataset and name it as df
HKTVMall_dataset = pd.read_csv(InProductsFile)

In [5]:
HKTVMall_dataset

,co,category,extract_date,band,weight,price,promotion,stock_status,URL
0,HKTVMall,vodka,20211231,Absolut 絕對伏特加 - 伏特加,750亳升,聖誕狂歡價,$ 118.00,stock on hand,https://www.hktvmall.com/hktv/zh/main/main/Win...
1,HKTVMall,vodka,20211231,Smirnoff Ice - Smirnoff Ice 伏特加檸檬汽酒 - 300ml x 6樽,300毫升 x 6,原價:$101.4│全店最少85折:,$ 60.00,stock on hand,https://www.hktvmall.com/hktv/zh/main/main/%E5...
2,HKTVMall,vodka,20211231,Baileys - Baileys 百利愛爾蘭忌廉甜酒--1000ml(香港行貨),1000ml,$ 328.00,$ 132.00,stock on hand,https://www.hktvmall.com/hktv/zh/main/main/BAR...
3,HKTVMall,vodka,20211231,Bombay Sapphire - 藍寶石邦比氈酒-1000ml,1000ml,$ 398.00,$ 167.00,stock on hand,https://www.hktvmall.com/hktv/zh/main/main/BAR...
4,HKTVMall,vodka,20211231,Absolut 絕對伏特加 - Vodka 1公升容量,1000毫升,開心消費 $152 飲靚酒 |原價$228,$ 152.00,stock on hand,https://www.hktvmall.com/hktv/zh/main/main/Win...
...,...,...,...,...,...,...,...,...,...
253,HKTVMall,vodka,20211231,Fever Tree - Fever Tree Soda 200mlx3,200mlX3,$ 90.00,$ 45.00,out of stock,https://www.hktvmall.com/hktv/zh/main/main/BAR...
254,HKTVMall,vodka,20211231,Bols - 波士杜松子酒湯力水套裝,NaN,$ 209.00,$ 149.00,out of stock,https://www.hktvmall.com/hktv/zh/main/main/BAR...
255,HKTVMall,vodka,20211231,Bosford - Rose Pink Gin,700ml,$ 169.00,$ 149.00,out of stock,https://www.hktvmall.com/hktv/zh/main/main/BAR...
256,HKTVMall,vodka,20211231,Bols - 士多啤梨味伏特加--750ml,750ml,$ 120.00,$ 69.00,out of stock,https://www.hktvmall.com/hktv/zh/main/main/BAR...


## delete records 

In [6]:
# price contain "全店至少
temp_db =  HKTVMall_dataset[~HKTVMall_dataset["price"].str.contains('"全店至少', na=False)]

In [7]:
# stock_status = 'Not found'
temp_db = temp_db[temp_db.stock_status != 'Not found']
temp_db.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 258 entries, 0 to 257
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   co            258 non-null    object
 1   category      258 non-null    object
 2   extract_date  258 non-null    int64 
 3   band          258 non-null    object
 4   weight        240 non-null    object
 5   price         258 non-null    object
 6   promotion     258 non-null    object
 7   stock_status  258 non-null    object
 8   URL           258 non-null    object
dtypes: int64(1), object(8)
memory usage: 20.2+ KB


In [8]:
print(len(temp_db))

258


## Start the data cleaning 

In [9]:
# promotion : remove{'$',',')
temp_db['promotion'] =temp_db.promotion.str.replace('$' , '')
temp_db['promotion'] =temp_db.promotion.str.replace(',' , '')

C:\Users\chan\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  


In [10]:
# create new fields
# new_price and set the type to object
temp_db['new_price'] =0
temp_db = temp_db.astype({'new_price': 'object'})

# new_band and set the type to object
temp_db['new_band'] =''
temp_db = temp_db.astype({'new_band': 'object'})

# new_description and set the type to object
temp_db['new_description'] =''
temp_db = temp_db.astype({'new_description': 'object'})

### splitting band into band and description

In [11]:
# new_band and set the type to object
temp_db['new_band'] =''
temp_db = temp_db.astype({'new_band': 'object'})
temp_db['new_description'] =''
temp_db = temp_db.astype({'new_description': 'object'})

In [12]:
#searchforband=' - '
#searchbandStr = temp_db.band[5449]
#print(searchbandStr)
#pos = find_str(searchbandStr, searchforband)
#new_band=searchbandStr[:pos]
#new_description=searchbandStr[pos+len(searchforband):]
#print (new_band)
#print(new_description)

In [13]:
temp_db

,co,category,extract_date,band,weight,price,promotion,stock_status,URL,new_price,new_band,new_description
0,HKTVMall,vodka,20211231,Absolut 絕對伏特加 - 伏特加,750亳升,聖誕狂歡價,118.00,stock on hand,https://www.hktvmall.com/hktv/zh/main/main/Win...,0,,
1,HKTVMall,vodka,20211231,Smirnoff Ice - Smirnoff Ice 伏特加檸檬汽酒 - 300ml x 6樽,300毫升 x 6,原價:$101.4│全店最少85折:,60.00,stock on hand,https://www.hktvmall.com/hktv/zh/main/main/%E5...,0,,
2,HKTVMall,vodka,20211231,Baileys - Baileys 百利愛爾蘭忌廉甜酒--1000ml(香港行貨),1000ml,$ 328.00,132.00,stock on hand,https://www.hktvmall.com/hktv/zh/main/main/BAR...,0,,
3,HKTVMall,vodka,20211231,Bombay Sapphire - 藍寶石邦比氈酒-1000ml,1000ml,$ 398.00,167.00,stock on hand,https://www.hktvmall.com/hktv/zh/main/main/BAR...,0,,
4,HKTVMall,vodka,20211231,Absolut 絕對伏特加 - Vodka 1公升容量,1000毫升,開心消費 $152 飲靚酒 |原價$228,152.00,stock on hand,https://www.hktvmall.com/hktv/zh/main/main/Win...,0,,
...,...,...,...,...,...,...,...,...,...,...,...,...
253,HKTVMall,vodka,20211231,Fever Tree - Fever Tree Soda 200mlx3,200mlX3,$ 90.00,45.00,out of stock,https://www.hktvmall.com/hktv/zh/main/main/BAR...,0,,
254,HKTVMall,vodka,20211231,Bols - 波士杜松子酒湯力水套裝,NaN,$ 209.00,149.00,out of stock,https://www.hktvmall.com/hktv/zh/main/main/BAR...,0,,
255,HKTVMall,vodka,20211231,Bosford - Rose Pink Gin,700ml,$ 169.00,149.00,out of stock,https://www.hktvmall.com/hktv/zh/main/main/BAR...,0,,
256,HKTVMall,vodka,20211231,Bols - 士多啤梨味伏特加--750ml,750ml,$ 120.00,69.00,out of stock,https://www.hktvmall.com/hktv/zh/main/main/BAR...,0,,


In [14]:
searchforband=' - '
for i in range(0,len(temp_db)):   
    record_no = temp_db.index[i]
    searchbandStr = temp_db.band[record_no]
    pos = find_str(searchbandStr, searchforband)
    new_band=searchbandStr[:pos]
    new_description=searchbandStr[pos+len(searchforband):]
    temp_db.new_band[record_no]=new_band
    temp_db.new_description[record_no]=new_description

C:\Users\chan\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\chan\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [15]:
temp_db.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 258 entries, 0 to 257
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   co               258 non-null    object
 1   category         258 non-null    object
 2   extract_date     258 non-null    int64 
 3   band             258 non-null    object
 4   weight           240 non-null    object
 5   price            258 non-null    object
 6   promotion        258 non-null    object
 7   stock_status     258 non-null    object
 8   URL              258 non-null    object
 9   new_price        258 non-null    object
 10  new_band         258 non-null    object
 11  new_description  258 non-null    object
dtypes: int64(1), object(11)
memory usage: 34.3+ KB


In [16]:
temp_db

,co,category,extract_date,band,weight,price,promotion,stock_status,URL,new_price,new_band,new_description
0,HKTVMall,vodka,20211231,Absolut 絕對伏特加 - 伏特加,750亳升,聖誕狂歡價,118.00,stock on hand,https://www.hktvmall.com/hktv/zh/main/main/Win...,0,Absolut 絕對伏特加,伏特加
1,HKTVMall,vodka,20211231,Smirnoff Ice - Smirnoff Ice 伏特加檸檬汽酒 - 300ml x 6樽,300毫升 x 6,原價:$101.4│全店最少85折:,60.00,stock on hand,https://www.hktvmall.com/hktv/zh/main/main/%E5...,0,Smirnoff Ice,Smirnoff Ice 伏特加檸檬汽酒 - 300ml x 6樽
2,HKTVMall,vodka,20211231,Baileys - Baileys 百利愛爾蘭忌廉甜酒--1000ml(香港行貨),1000ml,$ 328.00,132.00,stock on hand,https://www.hktvmall.com/hktv/zh/main/main/BAR...,0,Baileys,Baileys 百利愛爾蘭忌廉甜酒--1000ml(香港行貨)
3,HKTVMall,vodka,20211231,Bombay Sapphire - 藍寶石邦比氈酒-1000ml,1000ml,$ 398.00,167.00,stock on hand,https://www.hktvmall.com/hktv/zh/main/main/BAR...,0,Bombay Sapphire,藍寶石邦比氈酒-1000ml
4,HKTVMall,vodka,20211231,Absolut 絕對伏特加 - Vodka 1公升容量,1000毫升,開心消費 $152 飲靚酒 |原價$228,152.00,stock on hand,https://www.hktvmall.com/hktv/zh/main/main/Win...,0,Absolut 絕對伏特加,Vodka 1公升容量
...,...,...,...,...,...,...,...,...,...,...,...,...
253,HKTVMall,vodka,20211231,Fever Tree - Fever Tree Soda 200mlx3,200mlX3,$ 90.00,45.00,out of stock,https://www.hktvmall.com/hktv/zh/main/main/BAR...,0,Fever Tree,Fever Tree Soda 200mlx3
254,HKTVMall,vodka,20211231,Bols - 波士杜松子酒湯力水套裝,NaN,$ 209.00,149.00,out of stock,https://www.hktvmall.com/hktv/zh/main/main/BAR...,0,Bols,波士杜松子酒湯力水套裝
255,HKTVMall,vodka,20211231,Bosford - Rose Pink Gin,700ml,$ 169.00,149.00,out of stock,https://www.hktvmall.com/hktv/zh/main/main/BAR...,0,Bosford,Rose Pink Gin
256,HKTVMall,vodka,20211231,Bols - 士多啤梨味伏特加--750ml,750ml,$ 120.00,69.00,out of stock,https://www.hktvmall.com/hktv/zh/main/main/BAR...,0,Bols,士多啤梨味伏特加--750ml


### splitting the temp db into 4 temp db (temp_db2,temp_db3,temp_db4 and temp_db5) base on their criteria

In [17]:
# if match with searchfor, then set TrueFalse = 1
searchfor = ['原:$','原: $','原價$','原價 $','原價:$','原價：$','原價:','原價','RSP$',' | $','！！！','！！','！']
temp_db["TrueFalse"] = temp_db['price'].apply(lambda x: 1 if any(i in x for i in searchfor) else 0)

In [18]:
# find searchfor
temp_db2=temp_db[temp_db["TrueFalse"]==1].copy()
print(len(temp_db2))

29


In [19]:
# not find searchfor
temp_db3=temp_db[temp_db["TrueFalse"]==0].copy()
print(len(temp_db3))

229


In [20]:
# records with starts with '$'
temp_db4=temp_db3[temp_db3.price.str.startswith('$')].copy()
print(len(temp_db4))

196


In [21]:
# records for not starts with '$'
temp_db5=temp_db3[~temp_db3.price.str.startswith('$')].copy()
print(len(temp_db5))

#del temp_db3

33


### handling temp_db4 
remove{'$') from the price field

In [22]:
# remove{'$') from the price field
temp_db4['new_price']=temp_db4.price.str.replace('$' , '')
temp_db4['new_price']=temp_db4.new_price.str.replace(',' , '')

C:\Users\chan\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  


In [23]:
temp_db4

,co,category,extract_date,band,weight,price,promotion,stock_status,URL,new_price,new_band,new_description,TrueFalse
2,HKTVMall,vodka,20211231,Baileys - Baileys 百利愛爾蘭忌廉甜酒--1000ml(香港行貨),1000ml,$ 328.00,132.00,stock on hand,https://www.hktvmall.com/hktv/zh/main/main/BAR...,328.00,Baileys,Baileys 百利愛爾蘭忌廉甜酒--1000ml(香港行貨),0
3,HKTVMall,vodka,20211231,Bombay Sapphire - 藍寶石邦比氈酒-1000ml,1000ml,$ 398.00,167.00,stock on hand,https://www.hktvmall.com/hktv/zh/main/main/BAR...,398.00,Bombay Sapphire,藍寶石邦比氈酒-1000ml,0
6,HKTVMall,vodka,20211231,Stolichnaya - 蘇聯頂級伏特加 1公升,1公升,$ 136.00,98.00,stock on hand,https://www.hktvmall.com/hktv/zh/main/main/10%...,136.00,Stolichnaya,蘇聯頂級伏特加 1公升,0
7,HKTVMall,vodka,20211231,Windsor Gin - 溫沙 氈酒 Gin,1000ml,$ 100.00,55.00,stock on hand,https://www.hktvmall.com/hktv/zh/main/main/Yea...,100.00,Windsor Gin,溫沙 氈酒 Gin,0
8,HKTVMall,vodka,20211231,Jessy's Barrel - 德國伏特加 1000亳升 (1公升),1000亳升,$ 85.00,55.00,stock on hand,https://www.hktvmall.com/hktv/zh/main/main/10%...,85.00,Jessy's Barrel,德國伏特加 1000亳升 (1公升),0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,HKTVMall,vodka,20211231,Fever Tree - Fever Tree Soda 200mlx3,200mlX3,$ 90.00,45.00,out of stock,https://www.hktvmall.com/hktv/zh/main/main/BAR...,90.00,Fever Tree,Fever Tree Soda 200mlx3,0
254,HKTVMall,vodka,20211231,Bols - 波士杜松子酒湯力水套裝,NaN,$ 209.00,149.00,out of stock,https://www.hktvmall.com/hktv/zh/main/main/BAR...,209.00,Bols,波士杜松子酒湯力水套裝,0
255,HKTVMall,vodka,20211231,Bosford - Rose Pink Gin,700ml,$ 169.00,149.00,out of stock,https://www.hktvmall.com/hktv/zh/main/main/BAR...,169.00,Bosford,Rose Pink Gin,0
256,HKTVMall,vodka,20211231,Bols - 士多啤梨味伏特加--750ml,750ml,$ 120.00,69.00,out of stock,https://www.hktvmall.com/hktv/zh/main/main/BAR...,120.00,Bols,士多啤梨味伏特加--750ml,0


### handling temp_db2
temp_db2 : get the original price from the price field 

In [24]:
searchforprice = ['原:$','原: $','原價$','原價 $','原價:$','原價：$','原價:','原價','RSP$',' | $']
print(len(searchforprice))

10


In [25]:
temp_db2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29 entries, 1 to 234
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   co               29 non-null     object
 1   category         29 non-null     object
 2   extract_date     29 non-null     int64 
 3   band             29 non-null     object
 4   weight           28 non-null     object
 5   price            29 non-null     object
 6   promotion        29 non-null     object
 7   stock_status     29 non-null     object
 8   URL              29 non-null     object
 9   new_price        29 non-null     object
 10  new_band         29 non-null     object
 11  new_description  29 non-null     object
 12  TrueFalse        29 non-null     int64 
dtypes: int64(2), object(11)
memory usage: 3.2+ KB


In [26]:
# handling searchforprice
for i in range(0,len(temp_db2)):
    for j in range(0,len(searchforprice)):
        #print(searchforprice[j])
        record_no = temp_db2.index[i]
        searchStr = temp_db2.price[record_no]
        pos = find_str(searchStr, searchforprice[j])
        #print(pos)
        if pos !=-1:
            a_string = searchStr[pos:]
            numbers = re.findall('[0-9.]+', a_string)
            #print(numbers)
            temp_db2.new_price[record_no]=numbers[0]


# handling '！！！','！！','！'
# To find out which rows have NaNs in a new_price column
nan_rows = temp_db2[temp_db2['new_price'].isnull()]

for i in range(0,len(nan_rows)): 
    record_no = nan_rows.index[i]
    temp_db2.new_price[record_no]=temp_db2.promotion[record_no]
    temp_db2.promotion[record_no]=0


temp_db2['new_price'] =temp_db2.new_price.str.replace('$' , '')
temp_db2['new_price'] =temp_db2.new_price.str.replace(',' , '')

C:\Users\chan\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\chan\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.


In [27]:
temp_db2

,co,category,extract_date,band,weight,price,promotion,stock_status,URL,new_price,new_band,new_description,TrueFalse
1,HKTVMall,vodka,20211231,Smirnoff Ice - Smirnoff Ice 伏特加檸檬汽酒 - 300ml x 6樽,300毫升 x 6,原價:$101.4│全店最少85折:,60.00,stock on hand,https://www.hktvmall.com/hktv/zh/main/main/%E5...,101.4,Smirnoff Ice,Smirnoff Ice 伏特加檸檬汽酒 - 300ml x 6樽,1
4,HKTVMall,vodka,20211231,Absolut 絕對伏特加 - Vodka 1公升容量,1000毫升,開心消費 $152 飲靚酒 |原價$228,152.00,stock on hand,https://www.hktvmall.com/hktv/zh/main/main/Win...,228,Absolut 絕對伏特加,Vodka 1公升容量,1
12,HKTVMall,vodka,20211231,Grey Goose - 灰雁原味伏特加 750ml,750ML,聖誕狂歡價｜原價$450,382.00,stock on hand,https://www.hktvmall.com/hktv/zh/main/main/Nob...,450,Grey Goose,灰雁原味伏特加 750ml,1
15,HKTVMall,vodka,20211231,McCormick - Vodka 麥高魅 伏特加 四重蒸餾 Alc. 40% 1000ML,1000ML,聖誕狂歡價｜原價$89,59.00,stock on hand,https://www.hktvmall.com/hktv/zh/main/main/GD-...,89,McCormick,Vodka 麥高魅 伏特加 四重蒸餾 Alc. 40% 1000ML,1
21,HKTVMall,vodka,20211231,Voli - Vodka Raspberry Cocoa Fusion,750毫升,原價:$230|VIP優惠,78.00,stock on hand,https://www.hktvmall.com/hktv/zh/main/main/Win...,230,Voli,Vodka Raspberry Cocoa Fusion,1
33,HKTVMall,vodka,20211231,Finlandia - 酒辦 芬蘭迪加 伏特加 口感順滑 Finlandia Vodka A...,50ML,聖誕狂歡價｜原價$39,25.00,stock on hand,https://www.hktvmall.com/hktv/zh/main/main/GD-...,39,Finlandia,酒辦 芬蘭迪加 伏特加 口感順滑 Finlandia Vodka Alc. 40% 50ML,1
37,HKTVMall,vodka,20211231,JAMESON - Whiskey - 尊美醇愛爾蘭威士忌 Jameson Irish wh...,700ml,原價:348 / 世界上暢銷的愛爾蘭威士忌,288.00,stock on hand,https://www.hktvmall.com/hktv/zh/main/main/%E5...,348,JAMESON,Whiskey - 尊美醇愛爾蘭威士忌 Jameson Irish whiskey 700ml,1
52,HKTVMall,vodka,20211231,Holy Grass Vodka - Scottish Vodka,700ml,21% OFF | 原價 : $480,380.00,stock on hand,https://www.hktvmall.com/hktv/zh/main/main/%E5...,480,Holy Grass Vodka,Scottish Vodka,1
65,HKTVMall,vodka,20211231,市集啤酒 - 日本版 Smirnoff Ice - Original Vodka Mixed...,已售出 10+,曾經被紐約時報評為全球最好的伏特加！,33.00,stock on hand,https://www.hktvmall.com/hktv/zh/main/main/%E5...,NaN,市集啤酒,日本版 Smirnoff Ice - Original Vodka Mixed Drink ...,1
68,HKTVMall,vodka,20211231,Belvedere - Belvedere Vodka 700ML,700ML,聖誕狂歡價｜原價$500,358.00,stock on hand,https://www.hktvmall.com/hktv/zh/main/main/Nob...,500,Belvedere,Belvedere Vodka 700ML,1


### handling temp_db5
for the price is not start with '$', then price is replaced by promotion, and promotion will be set to zero

In [28]:
temp_db5['new_price']=temp_db5.promotion
temp_db5.promotion=0
#temp_db5

# combine all temp datasets: temp_db4, temp_db2 and temp_db5

In [29]:
frames = [temp_db4, temp_db2, temp_db5]
df_clean = pd.concat(frames)

In [30]:
df_clean

,co,category,extract_date,band,weight,price,promotion,stock_status,URL,new_price,new_band,new_description,TrueFalse
2,HKTVMall,vodka,20211231,Baileys - Baileys 百利愛爾蘭忌廉甜酒--1000ml(香港行貨),1000ml,$ 328.00,132.00,stock on hand,https://www.hktvmall.com/hktv/zh/main/main/BAR...,328.00,Baileys,Baileys 百利愛爾蘭忌廉甜酒--1000ml(香港行貨),0
3,HKTVMall,vodka,20211231,Bombay Sapphire - 藍寶石邦比氈酒-1000ml,1000ml,$ 398.00,167.00,stock on hand,https://www.hktvmall.com/hktv/zh/main/main/BAR...,398.00,Bombay Sapphire,藍寶石邦比氈酒-1000ml,0
6,HKTVMall,vodka,20211231,Stolichnaya - 蘇聯頂級伏特加 1公升,1公升,$ 136.00,98.00,stock on hand,https://www.hktvmall.com/hktv/zh/main/main/10%...,136.00,Stolichnaya,蘇聯頂級伏特加 1公升,0
7,HKTVMall,vodka,20211231,Windsor Gin - 溫沙 氈酒 Gin,1000ml,$ 100.00,55.00,stock on hand,https://www.hktvmall.com/hktv/zh/main/main/Yea...,100.00,Windsor Gin,溫沙 氈酒 Gin,0
8,HKTVMall,vodka,20211231,Jessy's Barrel - 德國伏特加 1000亳升 (1公升),1000亳升,$ 85.00,55.00,stock on hand,https://www.hktvmall.com/hktv/zh/main/main/10%...,85.00,Jessy's Barrel,德國伏特加 1000亳升 (1公升),0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,HKTVMall,vodka,20211231,哥頓 - Gin 哥頓氈酒 Gordon's London Dry Gin 1000ml,1000ml,"英國老牌子的GIN,價格親民",0,out of stock,https://www.hktvmall.com/hktv/zh/main/main/%E5...,168.00,哥頓,Gin 哥頓氈酒 Gordon's London Dry Gin 1000ml,0
238,HKTVMall,vodka,20211231,Russian Standard - 原味伏特加 [禮盒裝]1L #打邊爐#火鍋#cocktail,1L,聖誕優惠大放送!,0,out of stock,https://www.hktvmall.com/hktv/zh/main/main/Con...,228.00,Russian Standard,原味伏特加 [禮盒裝]1L #打邊爐#火鍋#cocktail,0
249,HKTVMall,vodka,20211231,Absolut 絕對伏特加 - ABSOLUT KURANT VODKA 750ML,已售出 20+,絕對(Absolut)是世界上暢銷的伏特加品牌之一,0,out of stock,https://www.hktvmall.com/hktv/zh/main/main/%E5...,180.00,Absolut 絕對伏特加,ABSOLUT KURANT VODKA 750ML,0
250,HKTVMall,vodka,20211231,阿佩羅 - 阿佩羅開胃酒 Aperol Liqueur 1000ml,1000ml,係意大利非常受歡迎餐前酒~,0,out of stock,https://www.hktvmall.com/hktv/zh/main/main/%E5...,148.00,阿佩羅,阿佩羅開胃酒 Aperol Liqueur 1000ml,0


In [31]:
# Remove the "price" and "TrueFalse" column
df_clean = df_clean.drop(['price','TrueFalse'],axis=1)

# Rename the dataset headers
df_clean = df_clean.rename({'new_price':'price'}, axis=1)
df_clean = df_clean.rename({'band':'old_band'}, axis=1)
df_clean = df_clean.rename({'new_band':'band'}, axis=1)
df_clean = df_clean.rename({'new_description':'description'}, axis=1)

# set the "price" and "promotion" column to float
df_clean = df_clean.astype({'price': float})
df_clean = df_clean.astype({'promotion': float})

# take a look on the dataset's info and attributes after changes
df_clean.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 258 entries, 2 to 251
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   co            258 non-null    object 
 1   category      258 non-null    object 
 2   extract_date  258 non-null    int64  
 3   old_band      258 non-null    object 
 4   weight        240 non-null    object 
 5   promotion     258 non-null    float64
 6   stock_status  258 non-null    object 
 7   URL           258 non-null    object 
 8   price         257 non-null    float64
 9   band          258 non-null    object 
 10  description   258 non-null    object 
dtypes: float64(2), int64(1), object(8)
memory usage: 24.2+ KB


In [32]:
# To find out which rows have NaNs in a price column
nan_rows = df_clean[df_clean['price'].isnull()]

for i in range(0,len(nan_rows)): 
    record_no = nan_rows.index[i]
    df_clean.price[record_no]=df_clean.promotion[record_no]
    df_clean.promotion[record_no]=0
    

# To find out which rows have zero     
nan_rows = df_clean[df_clean['price']==0]
for i in range(0,len(nan_rows)): 
    record_no = nan_rows.index[i]
    df_clean.price[record_no]=df_clean.promotion[record_no]
    df_clean.promotion[record_no]=0

C:\Users\chan\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\chan\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\chan\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\chan\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWar

In [33]:
# export the dataframe to csv file (1st time)
df_clean = df_clean[["co","category","extract_date","band","description","old_band","weight","price","promotion","stock_status","URL"]]

# sort the dataframe based on the index labels
df_clean.sort_index(axis = 0)

# export the final dataset to CSV file
df_clean.to_csv(OutProductsFile, index=False, encoding='utf_8_sig') 

In [34]:
#temp_db4.to_csv("output/HKTVMall_ProdInfo_4.csv", index=False, encoding='utf_8_sig') 

In [35]:
# delete all temp datasets
#del temp_db
#del temp_db2
#del temp_db3
#del temp_db4
#del temp_db5